In [1]:
from music21 import *
environment.Environment()['musicxmlPath'] = r'D:\Program Files\MuseScore 3\bin\MuseScore3.exe'
environment.Environment()['musescoreDirectPNGPath'] = r'D:\Program Files\MuseScore 3\bin\MuseScore3.exe'

In [3]:
PitchClass = ['C','D','E','F','G','A','B']
TypeClass = ['64th','32nd','16th','eighth','quarter','half','whole']   
class StrNote:
    '''
     对音符的字符串信息进行保存
     分别存储pitch，octave ,accidental，并最终拼接起来
    '''
    def __init__(self):
        '''
            默认简谱中的do是C5
            因此是pitch下标为0
            中音的八度octave是第五个
            没有升降号accidental 
            默认是四分之一音符4
            默认没有附点
        '''
        self.pitch = 0
        self.octave = 5
        self.accidental = ''
        self.type = 4
        self.dot = 0
        
    def __init__(self,pitch):
        '''
          有参构造
        '''
        self.pitch = pitch-1
        self.octave = 5
        self.accidental = ''
        self.type = 4
        self.dot = 0
            
    def __add__(self, y):
        '音调pitch加一位'
        self.pitch = (self.pitch+y)%7
        return self
        
    def __sub__(self,y):
        '音调pitch减一位'
        self.pitch = (self.pitch+7-y)%7
        return self
    
    def setflat(self):
        '降音'
        self.accidental = 'b'
    
    def setsharp(self):
        '升音'
        self.accidental = '#'
    
    def setnormal(self):
        '变成普通音'
        self.accidental = ''
        
    def higher(self):
        '提高一个八度'
        self.octave += 1 
        
    def lower(self):
        '降低一个八度'
        self.octave -= 1 
    
    def longer(self):
        '时间加长'
        if self.type == 5 and self.dot == 0:
            self.dot = 1
        else:
            self.type += 1
            
    def shotter(self):
        '时间减短'
        self.type -= 1
    
    def adddot(self):
        '添加附点'
        self.dot += 1
        
    def getstr(self):
        '返回这个音的字符串'
        return str(PitchClass[self.pitch]) + str(self.octave) + str(self.accidental)
        
    
    def getnote(self):
        if self.pitch == -1:
            stopnote = note.Rest(1)
            stopnote.duration.type = TypeClass[self.type]
            stopnote.duration.dots = self.dot
            return stopnote
        '返回音符对象'
        thisnote = note.Note(str(PitchClass[self.pitch]) + str(self.octave) + str(self.accidental))
        thisnote.duration.type = TypeClass[self.type]
        thisnote.duration.dots = self.dot
        return thisnote
 
'''判断是数字还是符号'''
def isnum(input):
    try:
        int(input)
        return True
    except:
        return False
'''
Summay:
    将字符串简谱转换成音乐流
Parameters:
    music_str - 简谱字符串  str
    play_it - 是否播放      bool
Return：
    s - Music21格式的音乐流 stream
'''
def str2stream(music_str,play_it = True):
    out_put = open('out.txt','w')
    notelist = []
    s = stream.Score(id='mainScore')
    '''
    lowerflag  -  低音标志()
    higherflag -  高音标志[]
    shotterflag - 短时标志<>
    longgerflag - 长时标志--
    
    '''
    lowerflag = False
    higherflag = False
    sharpflag = False
    shotterflag = 0
    measurenum = 1
    for each in music_str:
        #print(each)
        if isnum(each):
            midnote = StrNote(int(each))
            if lowerflag == True:
                midnote.lower()
            if higherflag == True:
                midnote.higher()
            if shotterflag != 0:
                for i in range(shotterflag):
                    midnote.shotter()
            if sharpflag == True:
                midnote.setsharp()
                #print(midnote)
                sharpflag = False
            notelist.append(midnote)
        else:
            if each == '(':
                lowerflag = True
            elif each == ')':
                lowerflag = False
            elif each == '[':
                higherflag = True 
            elif each == ']':
                higherflag = False 
            elif each == '<':
                shotterflag += 1 
            elif each == '>':
                shotterflag -= 1
            elif each == '.':
                notelist[len(notelist)-1].adddot()
            elif each == '-':
                notelist[len(notelist)-1].longer()
            elif each == '#':
                #sharpflag = True
                notelist[len(notelist)-1].setsharp()
            elif each == 'b':
                notelist[len(notelist)-1].setflat()
            elif each == '|':
                midstream = stream.Measure(number=measurenum)
                measurenum+=1
                for eachnote in notelist:
                    out_put.write(eachnote.getstr()+'\n')
                    midstream.append(eachnote.getnote())
                
                s.append(midstream)
                notelist.clear()
            else:
                continue
    
    if len(notelist)!= 0:
        midstream = stream.Measure(number=measurenum)
        for eachnote in notelist:
            out_put.write(eachnote.getstr()+'\n')
            midstream.append(eachnote.getnote())
        s.append(midstream)

        notelist.clear()
    if play_it == True:play(s)
    out_put.close()
    return s


def str2tone(music_str):
    out_put = open('out.txt','w')
    notelist = []
    s = stream.Score(id='mainScore')
    '''
    lowerflag  -  低音标志()
    higherflag -  高音标志[]
    shotterflag - 短时标志<>
    longgerflag - 长时标志--
    
    '''
    lowerflag = False
    higherflag = False
    sharpflag = False
    shotterflag = 0
    measurenum = 1
    for each in music_str:
        #print(each)
        if isnum(each):
            midnote = StrNote(int(each))
            if lowerflag == True:
                midnote.lower()
            if higherflag == True:
                midnote.higher()
            if shotterflag != 0:
                for i in range(shotterflag):
                    midnote.shotter()
            if sharpflag == True:
                midnote.setsharp()
                #print(midnote)
                sharpflag = False
            notelist.append(midnote)
        else:
            if each == '(':
                lowerflag = True
            elif each == ')':
                lowerflag = False
            elif each == '[':
                higherflag = True 
            elif each == ']':
                higherflag = False 
            elif each == '<':
                shotterflag += 1 
            elif each == '>':
                shotterflag -= 1
            elif each == '.':
                notelist[len(notelist)-1].adddot()
            elif each == '-':
                notelist[len(notelist)-1].longer()
            elif each == '#':
                #sharpflag = True
                notelist[len(notelist)-1].setsharp()
            elif each == 'b':
                notelist[len(notelist)-1].setflat()
    
    return notelist[0].getstr()

In [14]:
test_note_list = []
str_note = ['1','2','3','4','5','6','7']
# 普通数据音调
for note in str_note:
    test_note_list.append(note)
    
# 升音
for note in str_note:
    test_note_list.append(note+'#')
    
# 降音
for note in str_note:
    test_note_list.append(note+'b')

# 高音
for note in str_note:
    test_note_list.append('['+note+']')

# 低音
for note in str_note:
    test_note_list.append('('+note+')')
    
# 高音升音
for note in str_note:
    test_note_list.append('['+note+'#]')
    
# 高音降音
for note in str_note:
    test_note_list.append('['+note+'b]')
    
# 低音升音
for note in str_note:
    test_note_list.append('('+note+'#)')
    
# 低音降音
for note in str_note:
    test_note_list.append('('+note+'b)')

In [15]:
str(test_note_list)

"['1', '2', '3', '4', '5', '6', '7', '1#', '2#', '3#', '4#', '5#', '6#', '7#', '1b', '2b', '3b', '4b', '5b', '6b', '7b', '[1]', '[2]', '[3]', '[4]', '[5]', '[6]', '[7]', '(1)', '(2)', '(3)', '(4)', '(5)', '(6)', '(7)', '[1#]', '[2#]', '[3#]', '[4#]', '[5#]', '[6#]', '[7#]', '[1b]', '[2b]', '[3b]', '[4b]', '[5b]', '[6b]', '[7b]', '(1#)', '(2#)', '(3#)', '(4#)', '(5#)', '(6#)', '(7#)', '(1b)', '(2b)', '(3b)', '(4b)', '(5b)', '(6b)', '(7b)']"

In [16]:
test_ans_list = []
for note in test_note_list:
    test_ans_list.append(str2tone(note))

In [17]:
str(test_ans_list)

"['C5', 'D5', 'E5', 'F5', 'G5', 'A5', 'B5', 'C5#', 'D5#', 'E5#', 'F5#', 'G5#', 'A5#', 'B5#', 'C5b', 'D5b', 'E5b', 'F5b', 'G5b', 'A5b', 'B5b', 'C6', 'D6', 'E6', 'F6', 'G6', 'A6', 'B6', 'C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C6#', 'D6#', 'E6#', 'F6#', 'G6#', 'A6#', 'B6#', 'C6b', 'D6b', 'E6b', 'F6b', 'G6b', 'A6b', 'B6b', 'C4#', 'D4#', 'E4#', 'F4#', 'G4#', 'A4#', 'B4#', 'C4b', 'D4b', 'E4b', 'F4b', 'G4b', 'A4b', 'B4b']"

In [18]:
for i in range(len(test_ans_list)):
    print(test_note_list[i],test_ans_list[i])

1 C5
2 D5
3 E5
4 F5
5 G5
6 A5
7 B5
1# C5#
2# D5#
3# E5#
4# F5#
5# G5#
6# A5#
7# B5#
1b C5b
2b D5b
3b E5b
4b F5b
5b G5b
6b A5b
7b B5b
[1] C6
[2] D6
[3] E6
[4] F6
[5] G6
[6] A6
[7] B6
(1) C4
(2) D4
(3) E4
(4) F4
(5) G4
(6) A4
(7) B4
[1#] C6#
[2#] D6#
[3#] E6#
[4#] F6#
[5#] G6#
[6#] A6#
[7#] B6#
[1b] C6b
[2b] D6b
[3b] E6b
[4b] F6b
[5b] G6b
[6b] A6b
[7b] B6b
(1#) C4#
(2#) D4#
(3#) E4#
(4#) F4#
(5#) G4#
(6#) A4#
(7#) B4#
(1b) C4b
(2b) D4b
(3b) E4b
(4b) F4b
(5b) G4b
(6b) A4b
(7b) B4b


In [21]:
import json
test_dict  = {}
for i in range(len(test_ans_list)):
    testname = 'test{0}'.format(i)
    test_dict[testname] = {'note_str':test_note_list[i],'note_ans':test_ans_list[i]}
#dumps 将数据转换成字符串
json_str = json.dumps(test_dict,indent=4)# 使用indent格式化输出

In [23]:
with open(r"test_file.json","w") as f:
     f.write(json_str)
     print("保存json完成")

保存json完成


In [27]:
len(a)

63

In [26]:
import pytest
import json

# 读取数据
testname_list = []
for i in range(len(test_ans_list)):
    testname_list.append('test{0}'.format(i))
json_file = open("test_file.json",'r',encoding='gbk')  
a = json.load(json_file)

# 进行测试
@pytest.mark.parametrize('test_name', testname_list)
def test_str2note(test_name):
    print(a[test_name])